In [1]:
using CSV, DataFrames, Dates
using FileIO
# using LaTeXStrings

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:]

Row,Datetime,Latitude,Longitude,Depth,Magnitude
,DateTime,Float64,Float64,Float64,Float64
1,1976-02-03T13:29:16.400,45.32,25.99,184.3,2.6
2,1976-08-19T19:03:00.800,45.54,26.37,162.0,2.3
3,1976-09-07T17:38:08.200,45.62,26.5,155.3,3.6
4,1976-10-01T17:50:43.200,45.68,26.49,146.0,6.0
5,1976-10-15T10:54:48,47.1,25.6,17.0,3.9
6,1977-03-04T19:21:54.100,45.77,26.76,94.0,7.4
7,1977-03-04T21:21:01.100,45.22,26.65,141.0,3.0
8,1977-03-04T21:28:07.400,45.23,26.87,22.0,1.5
9,1977-03-04T23:17:14,45.0,27.2,33.0,2.5


In [4]:
mapcoords= Dict("minLat"=>minimum(romania.Latitude),
            "maxLat"=>maximum(romania.Latitude),
            "minLon"=>minimum(romania.Longitude),
            "maxLon"=>maximum(romania.Longitude),
            "minDepth"=>minimum(romania.Depth),
            "maxDepth"=>maximum(romania.Depth),
            "minMag"=>minimum(romania.Magnitude),
            "maxMag"=>maximum(romania.Magnitude)
)

Dict{String, Float64} with 8 entries:
  "maxLon"   => 29.84
  "minMag"   => 0.1
  "minLon"   => 20.19
  "maxMag"   => 7.4
  "minLat"   => 43.5941
  "maxLat"   => 48.23
  "maxDepth" => 218.4
  "minDepth" => 0.0

In [23]:
# specify the path to your local image file
img_path = "../maps/romania_wikipedia.png"
img = load(img_path);

In [99]:
using LaTeXStrings

In [8]:
using GLMakie

In [150]:
# Theme
seismic_theme = Theme(
    # fontsize = 24,
    Axis3 = (
        xlabelsize = 24,
        xlabeloffset = 40,
        xticklabelsize  = 16,

        ylabelsize = 24,
        ylabeloffset = 40,
        yticklabelsize  = 16,

        zlabelsize = 24,
        zlabeloffset = 40,
        zticklabelsize  = 16
        )
                    )

Attributes with 1 entry:
  Axis3 => Attributes with 9 entries:
    xlabeloffset => 40
    xlabelsize => 24
    xticklabelsize => 16
    ylabeloffset => 40
    ylabelsize => 24
    yticklabelsize => 16
    zlabeloffset => 40
    zlabelsize => 24
    zticklabelsize => 16

In [152]:
GLMakie.activate!()
set_theme!(seismic_theme)
# Makie.inline!(true)

In [153]:
Makie.current_default_theme()[:fonts]

Attributes with 4 entries:
  bold => TeX Gyre Heros Makie Bold
  bold_italic => TeX Gyre Heros Makie Bold Italic
  italic => TeX Gyre Heros Makie Italic
  regular => TeX Gyre Heros Makie

In [154]:
fig = Figure( resolution=(1080, 720))


In [159]:
empty!(fig)

In [160]:
ax = Axis3(fig[1,1];    
    title = L"\text{Seismic Events in Romania}",
    titlesize = 40,
    xlabel = "Longitude",
    ylabel = "Latitude",
    zlabel = "Depth",
    perspectiveness=0.5
)

Axis3()

In [161]:
# markerSizes = [10+log10(10*x) for x in romania.Magnitude];
markerSizes2 = [1.7^x for x in romania.Magnitude];

In [162]:
sc = scatter!(ax,
    romania.Longitude,
    romania.Latitude,
    -romania.Depth;
    markersize= markerSizes2,
    color= romania.Magnitude,
    colormap= (Reverse(:devon), 0.99),
    strokewidth= 0.5,
    shading= false
    # transparency= true
    # alpha = 0.8
)

Scatter{Tuple{Vector{Point{3, Float32}}}}

In [163]:
Colorbar(fig[1, 2], 
    sc, 
    label="Magnitudes", 
    height=Relative(0.5)
)

Colorbar()

In [164]:
image!(ax , 
    mapcoords["minLon"] .. mapcoords["maxLon"],
    mapcoords["minLat"] .. mapcoords["maxLat"],
    rotr90(img*0.6); 
    # img*0.6;
    transformation=(:xy, -mapcoords["maxDepth"]),
    # transformation=(:xy, 10),
)

Image{Tuple{Vector{Float32}, Vector{Float32}, Matrix{ColorTypes.RGBA{Float32}}}}

In [18]:
println(mapcoords["minLat"]," ",mapcoords["maxLat"]," ",mapcoords["minLon"]," ",mapcoords["maxLon"],)

43.5941 48.23 20.19 29.84


In [168]:
save("../results/romania_seismic_scatter.png", fig)